# Merge All Data

Merge air quality, AEMET climate and ERA5-Land data for each station.

The whole curation process have been performed using the `src/` scripts and this notebook only show examples of the process for monitoring sites in Madrid capital.

In [3]:
# Load packages
suppressMessages(library(tidyverse))
suppressMessages(library(lubridate))

In [8]:
# Working directory
setwd("AirQualityCOVID/")

# Source some scripts 
source("src/functions.R")

### Main Variables

In [9]:
start_dt <- ymd_hms("2013-01-01 00:00:00")
end_dt <- ymd_hms("2020-12-31 00:00:00")

## Stations Information

In [20]:
#-----------------------------
#    Air Quality Stations
#-----------------------------

sites.AQ <- read.csv("data/curation/checked_AQ.csv",
                    stringsAsFactor=T) %>%
                filter(Municipio == "Madrid") %>%
                mutate(site=as.character(site))

#-----------------------------
#       AEMET Stations
#-----------------------------

sites.AEMET <- read.csv("data/curation/checked_AEMET.csv",
                        stringsAsFactor=F, colClasses=c("indicativo"="character")) %>%
                filter(siteAQ %in% sites.AQ$site)

#-----------------------------
#       WorldMet Stations
#-----------------------------

sites.NOAA <- read.csv("data/curation/checked_NOAA-ISD.csv",
                     stringsAsFactor=T) %>%
                filter(siteAQ %in% sites.AQ$site)

In [21]:
head(sites.AQ)

,site,variable,site_name,latitude,longitude,elevation,country,site_type,site_area,date_start,date_end,Municipio,Poblaci..n,Estaci..n.tr..fico
,<chr>,<fct>,<fct>,<dbl>,<dbl>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>
1,es0118a,no,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19 00:00:00,2021-05-10 19:00:00,Madrid,3266126,ESCUELASAGUIRRE
2,es0118a,no2,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19 00:00:00,2021-05-10 19:00:00,Madrid,3266126,ESCUELASAGUIRRE
3,es0118a,o3,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19 00:00:00,2021-05-10 19:00:00,Madrid,3266126,ESCUELASAGUIRRE
4,es0118a,pm10,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19 00:00:00,2021-05-10 19:00:00,Madrid,3266126,ESCUELASAGUIRRE
5,es0118a,pm2.5,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19 00:00:00,2021-05-10 19:00:00,Madrid,3266126,ESCUELASAGUIRRE
6,es0120a,no,RAM<93>N Y CAJAL,40.45167,-3.677222,708,spain,traffic,urban,2002-01-01 00:00:00,2021-05-10 19:00:00,Madrid,3266126,RAM<93>NYCAJAL


## n-Table 

Table with all the relations between Air Quality stations with AEMET and WorldMet stations code

| siteAQ  | AEMET |   WorldMet   |
|---------|-------|--------------|
| es1580a | 1111  | 080210-99999 |
| es0118a | 3195  | 082230-99999 |
| es1438a | 0201D | 081810-99999 |
| es1340a | 4642E | 083830-99999 |

In [23]:
nn.stations <- data.frame()

for (st in levels(as.factor(sites.AQ$site))) {
    nn.stations <- rbind(nn.stations,
                         data.frame(siteAQ = as.factor(st),
                                    AEMET = as.factor(sites.AEMET[sites.AEMET$siteAQ == st,
                                                                  "indicativo"]),
                                    NOAA = sites.NOAA[sites.NOAA$siteAQ == st,
                                                              "code"]
                                   )
                        )
}

head(nn.stations)

,siteAQ,AEMET,NOAA
,<fct>,<fct>,<fct>
1,es0118a,3195,082230-99999
2,es0120a,3195,082210-99999
3,es1426a,3195,082210-99999
4,es1521a,3195,082210-99999
5,es1525a,3195,082230-99999
6,es1938a,3195,082230-99999


## ERA5-Land Data

In [24]:
#-----------------------------
#    ERA5-Land Data
#-----------------------------

folder.ERA5.Land <- "data/curation/ERA5-Land/"

# Relative Humidity
load(paste(folder.ERA5.Land,
           "rh_daily_2010_2020_final_stations.rda", sep=""))

rh.ERA5.Land <- data.as.datetime(df, "dates", "ymd") %>%
                    add_column(variable="RH", .after="dates")

# Solar radiation
load(paste(folder.ERA5.Land, 
           "ssrd_daily_2010_2020_final_stations.rda", sep=""))

ssrd.ERA5.Land <- data.as.datetime(df, "dates", "ymd") %>%
                    add_column(variable="solar.radiation", .after="dates")

ERA5.Land <- rbind(rh.ERA5.Land, ssrd.ERA5.Land)

## Merge Process

In [25]:
# Folder wherever take downloaded data, if it exists
Mto.files <- "data/curation/"

data_Mto <- data.frame()

In [27]:
for (st in levels(nn.stations$siteAQ)) {
    
    if (st %in% names(ERA5.Land)) {

        code <- nn.stations[nn.stations$siteAQ == st, "NOAA"]

        data.NOAA <- read.csv(paste(Mto.files, "NOAA-ISD/",
                                     code, ".csv", sep=""), stringsAsFactor=F) %>%
                        data.as.datetime("date", "ymd") %>%
                        select(-"code")

        indicativo <- nn.stations[nn.stations$siteAQ == st, "AEMET"]

        data.AEMET <- read.csv(paste(Mto.files, "AEMET/",
                                     indicativo, ".csv", sep=""), stringsAsFactor=F) %>%
                        data.as.datetime("fecha", "ymd") %>%
                        select("fecha", "tmed", "prec", 
                               "tmin", "tmax", "presMax", "presMin"
                               )

        data.row <- merge(x = data.NOAA, y = data.AEMET,
                            by.x = "date", by.y = "fecha", all = TRUE)

        for (vr in levels(as.factor(ERA5.Land$variable))) {
            data.row <- merge(x = data.row, 
                              y = ERA5.Land[ERA5.Land$variable == vr,
                                            c("dates", st)],
                              by.x = "date", by.y = "dates", all.x = TRUE
                             )
            names(data.row)[ncol(data.row)] <- vr
        }

        data.row[, "site"] <- st
        data_Mto <- rbind(data_Mto, data.row)
    }
}

head(data.row)